## Imports

In [1]:
from os import walk
from os.path import join

import pandas as pd
import numpy as np
import spacy
import nltk
import re
from bs4 import BeautifulSoup

from nltk.tokenize.toktok import ToktokTokenizer

## Files

In [2]:
Example_file = 'aclImdb/test/neg/16_1.txt'

test_neg = 'aclImdb/test/neg/'
test_pos = 'aclImdb/test/pos/'
train_neg = 'aclImdb/train/neg/'
train_pos = 'aclImdb/train/pos/'

In [225]:
# Testing files
stream = open(Example_file)
message = stream.read()
stream.close()
print(message)

For years, I've been a big fan of Park's work and "Old boy" is one of my all-times favorite.<br /><br />With lots of expectation I rented this movie, only to find the worst movie I've watched in awhile. It's not a proper horror movie; there's no suspense in it and even the "light" part is so lame, that I didn't know whether to laugh or cry.<br /><br />I introduced my younger brother to Chan-Wook Park and what a disappointment he got from this. For me, an idol has fallen.<br /><br />If you loved movies like "Old boy", the Mr & Lady "Vengeance" or even his short films on "Three extremes", don't waste your time, the film's not worth it.


In [226]:
def generate_msg(path):
    
    for root, dirnames, filenames in walk(path):
        for file_name in filenames:
            
            filepath = join(root, file_name)

            stream = open(filepath, encoding="utf8")
            message = stream.read()
            stream.close()
            
            yield message

In [227]:
def df_from_dir(path, classification):
    rows = []
    row_names = []
    
    for review in generate_msg(path):
        rows.append({'sentiment': classification, 'Review': review})
        
    return pd.DataFrame(rows)

In [245]:
reviews_s2 = df_from_dir(test_neg, '0')
reviews_s.tail()

,CATEGORY,Review
12495,positive,"Seeing as the vote average was pretty low, and..."
12496,positive,"The plot had some wretched, unbelievable twist..."
12497,positive,I am amazed at how this movie(and most others ...
12498,positive,A Christmas Together actually came before my t...
12499,positive,Working-class romantic drama from director Mar...


In [246]:
reviews_s2.shape

(12500, 2)

In [247]:
reviews_s2 = reviews_s2.append(df_from_dir(test_pos, '1'))

In [248]:
reviews_s2.shape

(25000, 2)

In [249]:
reviews_s2 = reviews_s2.append(df_from_dir(train_neg, '0'))

In [250]:
reviews_s2 = reviews_s2.append(df_from_dir(train_pos, '1'))

In [251]:
reviews_s2.shape

(50000, 2)

In [255]:
reviews_s2 = reviews_s2.sample(frac=1, random_state=42)

In [260]:
reviews_s2 = reviews_s2.reset_index()

In [267]:
reviews_s2.dtypes

index         int64
sentiment    object
Review       object
dtype: object

In [270]:
reviews_s2.head()

,sentiment,Review
0,0,Imagine The Big Chill with a cast of twenty-so...
1,0,I'd have to say that I've seen worse Sci Fi Ch...
2,0,Director Fabio Barreto got a strange Academy N...
3,0,Pretty bad PRC cheapie which I rarely bother t...
4,1,This is a very intriguing short movie by David...


In [271]:
reviews_s2.to_csv('movie_reviews.csv')

In [7]:
df = pd.read_csv('movie_reviews.csv')

In [96]:
df.head()

,Unnamed: 0,sentiment,Review
0,0,0,Imagine The Big Chill with a cast of twenty-so...
1,1,0,I'd have to say that I've seen worse Sci Fi Ch...
2,2,0,Director Fabio Barreto got a strange Academy N...
3,3,0,Pretty bad PRC cheapie which I rarely bother t...
4,4,1,This is a very intriguing short movie by David...


## Model Building

### Unsupervised Learning -> Lexicon-based models

In [16]:
from contractions import CONTRACTION_MAP
import unicodedata

nlp = spacy.load('en', parse = False, tag=False, entity=False)
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

import text_normalizer as tn

In [78]:
nltk.download('all', halt_on_error=False)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\brown.zip.
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_

[nltk_data]    |   Unzipping corpora\pros_cons.zip.
[nltk_data]    | Downloading package qc to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\qc.zip.
[nltk_data]    | Downloading package reuters to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package rte to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\rte.zip.
[nltk_data]    | Downloading package semcor to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package senseval to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\senseval.zip.
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\sentiwordnet.zip.
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |   

[nltk_data]    |   Unzipping corpora\nonbreaking_prefixes.zip.
[nltk_data]    | Downloading package vader_lexicon to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package porter_test to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping stemmers\porter_test.zip.
[nltk_data]    | Downloading package wmt15_eval to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping models\wmt15_eval.zip.
[nltk_data]    | Downloading package mwa_ppdb to
[nltk_data]    |     C:\Users\ddmas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping misc\mwa_ppdb.zip.
[nltk_data]    | 
[nltk_data]  Done downloading collection all


True

In [17]:
%%time
reviews = np.array(df['Review'])
sentiments = np.array(df['sentiment'])

# extract data for model evaluation
test_reviews = reviews[:30000]
test_sentiments = sentiments[:30000]
sample_review_ids = [999, 3533, 13010]

# normalize dataset
norm_test_reviews = tn.normalize_corpus(test_reviews)

Wall time: 50min 34s


In [18]:
test_reviews.shape

(30000,)

In [ ]:
np.savetxt

### AFINN

In [4]:
from afinn import Afinn

afn = Afinn(emoticons=True) 

In [32]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    print('Predicted Sentiment polarity:', afn.score(review))
    print('-'*60)

REVIEW: This film is worthwhile despite what you may hear. The performance of Marie Dressler (I hope I am spelling it right) as a drunken old sot is reason enough to see this film. It is an amazing performance. She is in a drunken stupor in three scenes for a good long while and she never does the same thing twice. You can actually smell the alcohol when she is done. Amazing. And Greta of course speaks her first lines on film and shes great. The Eugene O'Neill story is solid and like most O'Neill stories, very deep and intense. This is not light entertainment but if you appreciate those great character actors from the 30's and 40's you will like it. Some of the film is technically fuzzy but all in all worthwhile.
Actual Sentiment: 1
Predicted Sentiment polarity: 28.0
------------------------------------------------------------
REVIEW: Sigh. I'm baffled when I see a short like this get attention and assignments and whatnot. I saw this film at a festival before the filmmaker got any atte

In [20]:
%%time
sentiment_polarity = [afn.score(review) for review in test_reviews]
predicted_sentiments_AFINN = [1 if score >= 1.0 else 0 for score in sentiment_polarity]

Wall time: 8min 5s


### SentiWordNet

In [21]:
from nltk.corpus import sentiwordnet as swn

awesome = list(swn.senti_synsets('awesome', 'a'))[0]
print('Positive Polarity Score:', awesome.pos_score())
print('Negative Polarity Score:', awesome.neg_score())
print('Objective Score:', awesome.obj_score())

Positive Polarity Score: 0.875
Negative Polarity Score: 0.125
Objective Score: 0.0


In [22]:
def analyze_sentiment_sentiwordnet_lexicon(review,
                                           verbose=False):

    # tokenize and POS tag text tokens
    tagged_text = [(token.text, token.tag_) for token in tn.nlp(review)]
    pos_score = neg_score = token_count = obj_score = 0
    # get wordnet synsets based on POS tags
    # get sentiment scores if synsets are found
    for word, tag in tagged_text:
        ss_set = None
        if 'NN' in tag and list(swn.senti_synsets(word, 'n')):
            ss_set = list(swn.senti_synsets(word, 'n'))[0]
        elif 'VB' in tag and list(swn.senti_synsets(word, 'v')):
            ss_set = list(swn.senti_synsets(word, 'v'))[0]
        elif 'JJ' in tag and list(swn.senti_synsets(word, 'a')):
            ss_set = list(swn.senti_synsets(word, 'a'))[0]
        elif 'RB' in tag and list(swn.senti_synsets(word, 'r')):
            ss_set = list(swn.senti_synsets(word, 'r'))[0]
        # if senti-synset is found        
        if ss_set:
            # add scores for all found synsets
            pos_score += ss_set.pos_score()
            neg_score += ss_set.neg_score()
            obj_score += ss_set.obj_score()
            token_count += 1
    
    # aggregate final scores
    final_score = pos_score - neg_score
    norm_final_score = round(float(final_score) / token_count, 2)
    final_sentiment = 1 if norm_final_score >= 0 else 0
    if verbose:
        norm_obj_score = round(float(obj_score) / token_count, 2)
        norm_pos_score = round(float(pos_score) / token_count, 2)
        norm_neg_score = round(float(neg_score) / token_count, 2)
        # to display results in a nice table
        sentiment_frame = pd.DataFrame([[final_sentiment, norm_obj_score, norm_pos_score, 
                                         norm_neg_score, norm_final_score]],
                                       columns=['Predicted Sentiment', 'Objectivity', 'Positive', 'Negative', 'Overall'])
        print(sentiment_frame)
        
    return final_sentiment

In [33]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    pred = analyze_sentiment_sentiwordnet_lexicon(review, verbose=True)    
    print('-'*60)

REVIEW: This film is worthwhile despite what you may hear. The performance of Marie Dressler (I hope I am spelling it right) as a drunken old sot is reason enough to see this film. It is an amazing performance. She is in a drunken stupor in three scenes for a good long while and she never does the same thing twice. You can actually smell the alcohol when she is done. Amazing. And Greta of course speaks her first lines on film and shes great. The Eugene O'Neill story is solid and like most O'Neill stories, very deep and intense. This is not light entertainment but if you appreciate those great character actors from the 30's and 40's you will like it. Some of the film is technically fuzzy but all in all worthwhile.
Actual Sentiment: 1
   Predicted Sentiment  Objectivity  Positive  Negative  Overall
0                    1          0.8      0.12      0.07     0.05
------------------------------------------------------------
REVIEW: Sigh. I'm baffled when I see a short like this get attenti

In [24]:
%%time
predicted_sentiments_Senti = [analyze_sentiment_sentiwordnet_lexicon(review, verbose=False) for review in norm_test_reviews]

Wall time: 34min 26s


### VADER

In [25]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [26]:
def analyze_sentiment_vader_lexicon(review, 
                                    threshold=0.1,
                                    verbose=False):
    # pre-process text
    review = tn.strip_html_tags(review)
    review = tn.remove_accented_chars(review)
    review = tn.expand_contractions(review)
    
    # analyze the sentiment for review
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    # get aggregate scores and final sentiment
    agg_score = scores['compound']
    final_sentiment = 1 if agg_score >= threshold\
                                   else 0
    if verbose:
        # display detailed sentiment statistics
        positive = str(round(scores['pos'], 2)*100)+'%'
        final = round(agg_score, 2)
        negative = str(round(scores['neg'], 2)*100)+'%'
        neutral = str(round(scores['neu'], 2)*100)+'%'
        sentiment_frame = pd.DataFrame([[final_sentiment, final, positive,
                                        negative, neutral]],
                                        columns=['Predicted Sentiment', 'Polarity Score', 'Positive', 'Negative', 'Overall'])
        print(sentiment_frame)
    
    return final_sentiment

In [34]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    pred = analyze_sentiment_vader_lexicon(review, threshold=0.4, verbose=True)    
    print('-'*60)

REVIEW: This film is worthwhile despite what you may hear. The performance of Marie Dressler (I hope I am spelling it right) as a drunken old sot is reason enough to see this film. It is an amazing performance. She is in a drunken stupor in three scenes for a good long while and she never does the same thing twice. You can actually smell the alcohol when she is done. Amazing. And Greta of course speaks her first lines on film and shes great. The Eugene O'Neill story is solid and like most O'Neill stories, very deep and intense. This is not light entertainment but if you appreciate those great character actors from the 30's and 40's you will like it. Some of the film is technically fuzzy but all in all worthwhile.
Actual Sentiment: 1
   Predicted Sentiment  Polarity Score Positive Negative Overall
0                    1            0.98    22.0%     1.0%   77.0%
------------------------------------------------------------
REVIEW: Sigh. I'm baffled when I see a short like this get attenti

In [28]:
%%time
predicted_sentiments_VADER = [analyze_sentiment_vader_lexicon(review, threshold=0.4, verbose=False) for review in test_reviews]

Wall time: 14min 48s


## Evaluation

In [74]:
predicted_sentiments_AFINN_np = np.array(predicted_sentiments_AFINN)

In [88]:
predicted_sentiments_Senti_np = np.array(predicted_sentiments_Senti)

In [89]:
predicted_sentiments_VADER_np = np.array(predicted_sentiments_VADER)

In [95]:
np.savetxt('prediction_AFINN.txt', predicted_sentiments_AFINN_np)
np.savetxt('prediction_SentiWordNet.txt', predicted_sentiments_Senti_np)
np.savetxt('prediction_VADER.txt', predicted_sentiments_VADER_np)

In [75]:
predicted_sentiments_AFINN_np

array([1, 1, 0, ..., 0, 1, 1])

In [72]:
test_sentiments[8]

0

In [37]:
from sklearn.metrics import classification_report

In [77]:
true_pos = (test_sentiments == 1) & (predicted_sentiments_AFINN_np == 1)
true_pos.sum()

12637

In [78]:
confusion_matrix(test_sentiments, predicted_sentiments_AFINN_np)

array([[ 8676,  6396],
       [ 2291, 12637]], dtype=int64)

In [86]:
def evaluation_report(test, prediction):

    
    correct_docs = (test == prediction).sum()
    numdocs_wrong = test.shape[0] - correct_docs

    # Confusion Matrix
    true_pos = (test == 1) & (prediction == 1)
    false_pos = (test == 0) & (prediction == 1)
    false_neg = (test == 1) & (prediction == 0)
    true_neg = (test == 0) & (prediction == 0)
    
    
    # Accuracy
    fraction_wrong = numdocs_wrong/len(test)
    # Recall score
    recall_score = true_pos.sum() / (true_pos.sum() + false_neg.sum())
    # Precision Score
    precision_score = true_pos.sum() / (true_pos.sum() + false_pos.sum())
    # F1 Score
    f1_score = 2 * (precision_score * recall_score) / (precision_score + recall_score)

    
    # Printing the evaluation 
    print('Model Performance metrics: ')
    print('-'*20)
    print('Accuracy of the model is {:.2%}'.format(1-fraction_wrong))
    
    print('Recall score is {:.2%}'.format(recall_score))
    #print(re)
    print('Precision score is {:.3}'.format(precision_score))

    print('F1 Score is {:.2}'.format(f1_score))

    
    print('\n Model Classification report:')
    print('-'*20)
    target = ['positive', 'negative']
    print(classification_report(test, prediction, target_names=target))
    
    print('\n Prediction Confusion Matrix:')
    print('-'*20)
    confusion_matrix = pd.DataFrame([[true_pos.sum(), false_neg.sum()],[false_pos.sum().sum(),true_neg.sum()]],
                         columns=pd.MultiIndex(levels=[['predicted:'], 
                                                    ['positive', 'negative']], 
                                                    codes=[[0,0],[0,1]]),
                         index = pd.MultiIndex(levels=[['Actual:'], 
                                                    ['positive', 'negative']], 
                                                    codes=[[0,0],[0,1]]))
    print(confusion_matrix)

In [87]:
print("Evaluation of AFINN\n\n")
evaluation_report(test_sentiments, predicted_sentiments_AFINN_np)

Evaluation of AFINN


Model Performance metrics: 
--------------------
Accuracy of the model is 71.04%
Recall score is 84.65%
Precision score is 0.664
F1 Score is 0.74

 Model Classification report:
--------------------
              precision    recall  f1-score   support

    positive       0.79      0.58      0.67     15072
    negative       0.66      0.85      0.74     14928

    accuracy                           0.71     30000
   macro avg       0.73      0.71      0.71     30000
weighted avg       0.73      0.71      0.71     30000


 Prediction Confusion Matrix:
--------------------
                 predicted:         
                   positive negative
Actual: positive      12637     2291
        negative       6396     8676


In [92]:
print("Evaluation of Senti\n\n")
evaluation_report(test_sentiments, predicted_sentiments_Senti_np)

Evaluation of Senti


Model Performance metrics: 
--------------------
Accuracy of the model is 68.26%
Recall score is 74.37%
Precision score is 0.661
F1 Score is 0.7

 Model Classification report:
--------------------
              precision    recall  f1-score   support

    positive       0.71      0.62      0.66     15072
    negative       0.66      0.74      0.70     14928

    accuracy                           0.68     30000
   macro avg       0.69      0.68      0.68     30000
weighted avg       0.69      0.68      0.68     30000


 Prediction Confusion Matrix:
--------------------
                 predicted:         
                   positive negative
Actual: positive      11102     3826
        negative       5696     9376


In [93]:
print("Evaluation of VADER\n\n")
evaluation_report(test_sentiments, predicted_sentiments_VADER_np)

Evaluation of VADER


Model Performance metrics: 
--------------------
Accuracy of the model is 70.97%
Recall score is 82.78%
Precision score is 0.668
F1 Score is 0.74

 Model Classification report:
--------------------
              precision    recall  f1-score   support

    positive       0.78      0.59      0.67     15072
    negative       0.67      0.83      0.74     14928

    accuracy                           0.71     30000
   macro avg       0.72      0.71      0.71     30000
weighted avg       0.72      0.71      0.71     30000


 Prediction Confusion Matrix:
--------------------
                 predicted:         
                   positive negative
Actual: positive      12358     2570
        negative       6139     8933
